In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import DictionaryLearning
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load your data
# Load measurement file:
measurement = np.load('dataset/meas_symm_1.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs']




(9797, 15, 251, 2)


In [11]:
# selected_channel = [3, 6]
# true_labels = []

# for cir in range(data_cir.shape[0]):
#     for channel in selected_channel:
        
#         # Setting True label
#         if channel == 3:
#             # Legitimate user (Alice)
#             true_label = 1  
#         else:
#             # Illegitimate user (Eve)
#             true_label = -1
#         true_labels.append(true_label)
        
        
# Define channels
legit_channel = 3  # Channel 3 is legitimate
illegit_channel = 6  # Channel 6 is illegitimate

# Extract data for legitimate and illegitimate channels
X_legit = data_cir[:, legit_channel, :, :]  # Shape: (9797, 251, 2)
X_illegit = data_cir[:, illegit_channel, :, :]  # Shape: (9797, 251, 2)


# Create labels
y_legit = np.ones(X_legit.shape[0])  # Label as 1 for legit
print(y_legit.shape)
y_illegit = -1 * np.ones(X_illegit.shape[0])  # Label as -1 for illegit

# Combine data and labels
X_combined = np.concatenate([X_legit, X_illegit], axis=0)
y_combined = np.concatenate([y_legit, y_illegit], axis=0)

(9797,)


In [ ]:
# Assuming the labels are known and binary (1 for legitimate, -1 for illegitimate)
labels = measurement['labels']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape data for sparse representation (flatten channels and samples)
# Merging real and imaginary parts into a single dimension
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# Step 1: Learn dictionary on training data
n_components = 100  # Number of dictionary atoms
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', random_state=42)
X_train_sparse = dict_learner.fit_transform(X_train_reshaped)

# Step 2: Transform test data into sparse codes using learned dictionary
X_test_sparse = dict_learner.transform(X_test_reshaped)

# Step 3: Train a classifier on sparse codes
classifier = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=42))
classifier.fit(X_train_sparse, y_train)

# Step 4: Predict on test data
y_pred = classifier.predict(X_test_sparse)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))